# England vs Scotland

The dashboard below compares the vaccination rates in different age groups in England and Scotland. The rates are the percentage of the population who has had their second dose. 
The 'difference' value highlights the difference in vaccination rates between the two nations, with Scotland having vaccinated a greater proportion of its population.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Loading the JSON files of data to create default values for the graph.
jsondata={}
with open("eng.json", "rt") as IFILE:
    eng=json.load(IFILE)
engjson = eng['data'][0]
with open("scot.json", "rt") as IFILE:
    scot=json.load(IFILE)
scotjson = scot['data'][0]

In [ ]:
engjson

In [4]:
def wrangle_data(datascot,dataeng):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe. """
    def add_dicts(vaccined,ages):
        """Function to manipulate England's age ranges to align with Scotland, combines dictionaries of multiple age ranges into one to match Scotland.
        Parameters are raw data and the list of age ranges to combine."""
        agel = ages[:] #getting a new age range from the list of ages
        agel = [x.replace('_',' ').replace('+',' ') for x in agel]
        agel = ' '.join(agel)
        agel = [int(i) for i in agel.split()]
        agel.sort()
        if agel[-1]!=90:
            newval = f'{agel[0]}_{agel[-1]}' 
        else:
            newval = f'{agel[0]}+'
        newd = {k:0 for k in vaccined[0]} #create a new dict with merged values
        for d in vaccined: #iterating over the raw data to get the values to combine
            for key,value in d.items():
                d[key]=0 if d[key] is None else d[key]
            if d['age'] in ages: #finding the dictionary to combine
                for key, value in newd.items():
                    if key =='age':
                        newd['age']=newval
                    else:
                        value = float(d[key])
                        newd[key]+=value
                i = vaccined.index(d) #inserting the new dict into the original data and removing the original dict
                vaccined.pop(i)
                if newd not in vaccined:
                    vaccined[i]=newd
                else:
                    vaccined
        return vaccined #returning the list of data with the new dictionary inserted
    agesscot=[x['age'] for x in datascot['vaccinated']] #list of ages for x axis
    vaccineseng = dataeng['vaccinated'].copy() #copying the data to manipulate
    ages = [['18_24','25_29'], ['30_34','35_39'],['40_44','45_49'],['80_84','85_89','90+']]# lists of lists of ages to combine 
    for a in ages: #iterating over this list to call the manipulation function on each
        vaccineseng = add_dicts(vaccineseng,a)
    def min_age(agerange):
        agerange=agerange.replace('+','') # remove the + from 90+
        start=agerange.split('_')[0]
        return int(start)
    agesscot.sort(key=min_age)
    nations ='' #getting a list of nations for the plot values 
    for v in dataeng:
        if dataeng['nation'] not in nations:
            nations+=' '+dataeng['nation']
    for v in datascot:
        if datascot['nation'] not in nations:
            nations+=' '+datascot['nation']
    nations = nations.split()
    age_df=pd.DataFrame(index=agesscot, columns=nations+['Difference'])#adding in a difference column to show comparison
    for entry in datascot['vaccinated']: #iterating over dictionaries, each age band is a dictionary
        ageband=entry['age'] 
        age_df.loc[ageband, 'Scotland']=entry['cumVaccinationCompleteCoverageByVaccinationDatePercentage']
    for entry in vaccineseng: 
        ageband = entry['age']
        if ageband in agesscot: 
            percent = round((entry['cumPeopleVaccinatedCompleteByVaccinationDate']/entry['VaccineRegisterPopulationByVaccinationDate'])*100,1)#calculating this field for England due to manipulation above
            age_df.loc[ageband, 'England']= percent
    age_df['Difference']=age_df['Scotland']-age_df['England']
    return age_df

df = wrangle_data(scotjson,engjson) # wrangling the JSON data to ensure there is data at dashboard startup

Click the 'refresh data' button below to retrieve the most up-to-date data from Public Health England.

In [5]:
"""Callback functions to be called by refresh data button. Two API calls to retrieve data for England and Scotland"""
def access_eng(): #england data
    filters1 = [
        'areaType=nation',
        'areaName=england'
    ]
    structure1 = {
        'nation':'areaName',
        "vaccinated":"vaccinationsAgeDemographics"
    }
    apiE = Cov19API(filters=filters1, structure=structure1)
    england = apiE.get_json()
    apiE = england['data'][0] #formatting the same as JSON files
    apibutton.icon="check"
    apibutton.disabled=True
    return apiE # return data read from the API
def access_scot():
    filters2 = [
    'areaType=nation',
    'areaName=scotland'
    ]
    structure2 = {
    'nation':'areaName',
    "vaccinated":"vaccinationsAgeDemographics"
    }
    apiS = Cov19API(filters=filters2, structure=structure2)
    scotland = apiS.get_json()
    apiS = scotland['data'][0] #formatting the same as JSON files
    apibutton.icon="check"
    apibutton.disabled=True
    return apiS # return data read from the API


In [6]:
def api_button_callback(button):
    """ Button callback - takes the button as its parameter, calls the functions to access the API for Scotland and England data,
    wrangles the data and updates the dataframe for plotting """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apiscot=access_scot()
    apieng= access_eng()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apiscot,apieng)
    # this function simulates the interaction with widget to refresh the graph.
    refresh_graph()
    #changing the button to 'check' and disabling the button to avoid too many API calls by end user. 
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

apibutton.on_click(api_button_callback) # registers the button callback function with the button

display(apibutton)


Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

You can explore the graph below by selecting the nation whose rates you want to see, or the difference between them, in the 'Nation' menu. You can also select whether you want to view this data as a bar or line graph in the 'Graph Type' menu.

In [ ]:
nation=wdg.SelectMultiple(
    options=['England','Scotland','Difference'], # options available
    value=['England','Scotland','Difference'], # default value
    rows=3, 
    description='Nation',
    disabled=False
)

gtype=wdg.RadioButtons(
    options=['bar', 'line'],
    value='bar', # default value
    description='Graph Type',
    disabled=False
)

controls=wdg.HBox([nation, gtype])

def age_graph(graphcolumns,type):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0 and type == 'bar':
        df.plot(kind='bar', y=list(graphcolumns)) # changing graphcolumns tuple to a list
        plt.show() 
    elif ncols>0 and type =='line':
        df.plot(kind='line', y=list(graphcolumns)) # changing graphcolumns tuple to a list
        plt.show() 
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


def refresh_graph():
    current=nation.value #value is a tuple
    if current==(nation.options[0],):
        other=(nation.options[1],)
    else:
        other=(nation.options[0],)
    nation.value=other # forces the redraw
    nation.value=current # now we can change it back
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(age_graph, {'graphcolumns': nation,'type':gtype})

display(controls, output)

Output()

**Author and Copyright Notice** Author: Ellen Fitzgerald *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*